In [73]:
import json
from nltk.stem.porter import *
stemmer = PorterStemmer()
courses={}
with open('all_course_info.json') as json_file:  
    data = json.load(json_file)
    for p in data['courses']:
        wordlist=[]
        for word in p['Name'].split() + p['Area'].split() +["year"+p['Year']]:
            word=word.lower()
            word=stemmer.stem(word)
            wordlist.append(word)
        courses[p['Name']]=wordlist

In [74]:
print(courses)

{'Automated Reasoning': ['autom', 'reason', 'artifici', 'intellig', 'year3'], 'Computer Design': ['comput', 'design', 'architectur', 'year3'], 'Foundations of Natural Language Processing': ['foundat', 'of', 'natur', 'languag', 'process', 'artifici', 'intellig', 'year3'], 'Introduction to Vision and Robotics': ['introduct', 'to', 'vision', 'and', 'robot', 'artifici', 'intellig', 'year3'], 'Operating Systems': ['oper', 'system', 'architectur', 'year3'], 'System Design Project': ['system', 'design', 'project', 'softwar', 'engin', 'year3'], 'Algorithms and Data Structures': ['algorithm', 'and', 'data', 'structur', 'algorithm', 'and', 'data', 'structur', 'year3'], 'Computational Cognitive Science': ['comput', 'cognit', 'scienc', 'artifici', 'intellig', 'year3'], 'Computer Communications and Networks': ['comput', 'commun', 'and', 'network', 'network', 'year3'], 'Computer Security': ['comput', 'secur', 'secur', 'year3'], 'Database Systems': ['databas', 'system', 'data', 'scienc', 'year3'], 'C

In [76]:
search = input("Search for a course ")
search=search.lower()
search=search.split()
stemmed=[]
for word in range(0, len(search)):
    if(search[word] in ['1', '2', '3', '4', '5']):
        if(word>0):
            if(search[word-1]=="year"):
                stemmed[len(stemmed)-1]="year"+search[word]
            else:
                stemmed.append(stemmer.stem(search[word]))
        else:
            stemmed.append(stemmer.stem(search[word]))
    else:
        stemmed.append(stemmer.stem(search[word]))
search=stemmed
print(search)

Search for a course year 2
['year2']


In [77]:
results={}
for course in courses:
    matches=0
    for words in range(0, len(courses[course])):
        for query in range(0, len(search)):
            if search[query] in courses[course][words]:
                if(query<len(search)-1 and words < len(courses[course])-1):
                    if(search[query+1] in courses[course][words+1]):
                        matches+=1
                    if(search[query+1] == courses[course][words+1]):
                        matches+=1
                matches+=1
            if (search[query] ==courses[course][words]):
                if(query<len(search)-1 and words < len(courses[course])-1):
                    if(search[query+1] in courses[course][words+1]):
                        matches+=1
                    if(search[query+1] == courses[course][words+1]):
                        matches+=1
                matches+=1
    if(matches>0):
        if matches not in results:
            results[matches]=[]
        results[matches].append(course)

In [78]:
for output in reversed(sorted(results.keys())):
    for course in results[output]:
        print(course, output)

Discrete Mathematics and Mathematical Reasoning 2
Informatics 2A - Processing Formal and Natural Languages 2
Informatics 2B - Algorithms, Data Structures, Learning 2
Informatics 2C - Introduction to Computer Systems 2
Informatics 2C - Introduction to Software Engineering 2
Informatics 2D - Reasoning and Agents 2
